![Insper](https://github.com/danielscarvalho/Insper-DS-Dicas/blob/master/Insper-Logo.png?raw=true)

# Insper Pós-Graduação
## Programa Avançado em Data Science e Decisão [»](https://www.insper.edu.br/pos-graduacao/programas-avancados/programa-avancado-em-data-science-e-decisao/)


# Atividade Integradora
## Setup

### Dependências

In [ ]:
import pandas as pd
import numpy as np
from dfply import *
import altair as alt
import missingno as msno
from ydata_profiling import ProfileReport
import matplotlib
import matplotlib.pyplot as plt 
import math
import seaborn as sns
from sklearn import linear_model
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

### Carregamento dos Dados

Leitura da base de dados e do dicionários de dados

In [ ]:
data = pd.read_csv("cs_bisnode_panel.csv")
dicionario_de_dados_0 = pd.read_excel("bisnode_variable_names.xls", header=4)

Visualização inicial da base de dados

In [ ]:
data.head()

In [ ]:
data.describe()

## Dicionário de dados
### Limpeza

Ao carregar o dicionário de dados a primeira coluna pega seu nome da quarta linha da tabela (argumento `header=4` acima). As outras três colunas são nomeadas abaixo.

In [ ]:
dicionario_de_dados_1 = dicionario_de_dados_0.rename({'Unnamed: 1': 'description',
                                                      'Unnamed: 2': 'type',
                                                      'Unnamed: 3': 'footnote'},
                                                     axis=1)

In [ ]:
dicionario_de_dados_1.columns

Então, retiramos as linhas não relevantes para a análise, incluindo linhas totalmente em branco e uma linha com informação de versão da base de dados: 
 - `v 0.92. 2021-02-04`

In [ ]:
dicionario_de_dados = dicionario_de_dados_1\
                       .drop(index=54)\
                       .dropna(how="all")\
                       .reset_index()\
                       .drop('index', axis='columns')

In [ ]:
dicionario_de_dados.sample(5)

## Dados faltantes
---

###  Visualização
Vamos verificar dados faltando do banco de dados:

Criação de função para analisar os dados faltantes.

In [ ]:
def show_missing(df):
    """Return a Pandas dataframe describing the contents of a source dataframe including missing values."""
    
    variables = []
    dtypes = []
    count = []
    unique = []
    missing = []
    pc_missing = []
    
    for item in df.columns:
        variables.append(item)
        dtypes.append(df[item].dtype)
        count.append(len(df[item]))
        unique.append(len(df[item].unique()))
        missing.append(df[item].isna().sum())
        pc_missing.append(round((df[item].isna().sum() / len(df[item])) * 100, 2))

    output = pd.DataFrame({
        'variable': variables, 
        'dtype': dtypes,
        'count': count,
        'unique': unique,
        'missing': missing, 
        'pc_missing': pc_missing
    })    
        
    return output

Verificando dados com maior falta de informações:

In [ ]:
missing_data = show_missing(data).sort_values('pc_missing', ascending=False, ignore_index = True)

index_full_data = list(missing_data[missing_data['missing']==0].index)

missing_data.drop(labels=index_full_data, axis='index', inplace=True)

missing_data.head(10)

In [ ]:
missing_val_columns = missing_data['variable'][missing_data['pc_missing']>0]

In [ ]:
msno.bar(data[missing_val_columns], figsize=(16, 4))

### Colunas com poucos dados
Seis colunas tem mais de 90% de dados faltantes. Aqui removemos essas colunas. Removemos tambem as colunas "begin" e "end" que contêm informação sobre o período a que os dados dizem respeito. Consideraremos na análise essa mesma informação contida na variável "year".

In [ ]:
columns_to_remove = ['COGS',
                     'finished_prod',
                     'net_dom_sales',
                     'net_exp_sales',
                     'wages',
                     'D',
                     'begin',
                     'end'] 

data.drop(columns=columns_to_remove, inplace=True)

data.columns

As duas próximas variáveis com maior volume de dados faltantes são exit_year(86.50%) e exit_date (80.48%). Para as observações com informação em exit_date e não em exit_year, preenchemos exit_year com a informação de ano em exit_date. Como a presente análise tem granularidade anual, retiramos a coluna exit_date após esse tratamento.

In [ ]:
# Quando o exit_year está ausente, pega o ano de exit_date
data.loc[data['exit_year'].isna(), 'exit_year'] = pd.to_datetime(data['exit_date']).dt.year

# Substitui valores NaN em exit_year por "-"
data['exit_year'] = data['exit_year'].fillna('-')

# Exclui a coluna exit_date
data = data.drop(columns='exit_date')

### "founded_year"
Aproveitamos para tratar aqui a variável "founded_year" que utilizamos mais à frente na criação da feature "company_age".

In [ ]:
missing_data.loc[missing_data['variable'].isin(['founded_year','founded_date'])]

Convertemos 'founde_date' para datetime.\
**Nota:** A versão do código abaixo é mais complexa do que o necessário, mas generalizavel para tratar mais de uma variável de uma vez.

In [ ]:
colunas_data = ['founded_date']

data.dtypes[colunas_data]

In [ ]:
for column in colunas_data:
    data[column] = pd.to_datetime(data[column], format='%Y-%m-%d')

data[colunas_data].dtypes

De forma analoga ao tratamento de "exit_year" e "exit_date", utilizamos a variável "founded_date" (que tem uma menor proporção de valores faltantes) para preencher alguns dos valores faltantes em "founded_year" (que tem a granularidade desejada para a análise).

In [ ]:
assert any(data.loc[(data['founded_date'].isna() & 
                     data['founded_year'].isna()),
                    'comp_id'].count() ==\
           missing_data.loc[missing_data['variable'] == 'founded_date', 'missing'])

As observações sem informação em "founded_date" também não tem informação para "founded_year" e são retiradas.

In [ ]:
# Remove linhas onde 'founded_date' é NaN
data = data.dropna(subset=['founded_date'])

# Extrai o ano de 'founded_date' e substitui os valores ausentes em 'founded_year'
data['founded_year'] = data['founded_date'].dt.year

# Descarta a coluna 'founded_date', pois o ano de fundação da empresa é suficiente
data = data.drop(columns=['founded_date'])

### Checkpoint
Análise dos percentuais de missing após os tratamentos.

In [ ]:
missing_data2 = show_missing(data).sort_values('pc_missing', ascending=False, ignore_index = True)

index_full_data = list(missing_data2[missing_data2['missing']==0].index)

missing_data2.drop(labels=index_full_data, axis='index', inplace=True)

missing_data2

### Colunas com dados quase completos

Aqui tratamos as colunas com 5% ou menos dados faltantes. Primeiramente checamos se esses dados estão faltando nas mesmas observações ou se estão distribuidos em observações diferentes.

In [ ]:
# Lista de colunas com menos de 5% de dados ausentes

cols = list(missing_data2['variable'][missing_data2['pc_missing']<5.0])

# Verifica quais linhas possuem dados ausentes nessas colunas

missing_rows = data[cols].isnull().any(axis=1)      

# Calcula o percentual de linhas com dados ausentes

pc_missing_rows = 100 * missing_rows.sum() / len(data)

print(f'Percentual de linhas com dados ausentes em pelo menos uma das colunas mencionadas: {pc_missing_rows:.2f}%')

In [ ]:
# Remove variáveis com menos de 5% de dados ausentes
removable_na_columns = list(missing_data2['variable'][missing_data2['pc_missing']<5.0])

data.dropna(subset=removable_na_columns, inplace=True)

Valores faltantes após os tratamentos.

In [ ]:
missing_data_3 = show_missing(data).sort_values('pc_missing', ascending=False, ignore_index = True)

index_full_data = list(missing_data_3[missing_data_3['missing']==0].index)

missing_data_3.drop(labels=index_full_data, axis='index', inplace=True)

missing_data_3

In [ ]:
msno.matrix(data)

A tabela mostra 6 variaveis com cerca de 20% de dados faltantes nesse ponto da analise, e o grafico indica que essa ausencia de dados coincide nas mesmas observacoes. Optamos por nao retirar essas observacoes pois nao esperamos utilizar essas variaveis na analise.

### "labor_avg" e "birth_year"

In [ ]:
 # Plota um histograma para "labor_avg"
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.hist(data['labor_avg'].dropna(), bins=30, color='blue', alpha=0.7)
plt.title('Distribuição de labor_avg')
plt.xlabel('labor_avg')
plt.ylabel('Frequência')

# Plota um histograma para "birth year CEO"
plt.subplot(1, 2, 2)
plt.hist(data['birth_year'].dropna(), bins=30, color='green', alpha=0.7)
plt.title('Distribuição de birth year CEO')
plt.xlabel('birth year CEO')
plt.ylabel('Frequência')

plt.tight_layout()
plt.show()

In [ ]:
# Calcula e mostra a assimetria das variáveis
print(f"Assimetria de labor_avg: {data['labor_avg'].skew()}")
print(f"Assimetria de birth year CEO: {data['birth_year'].skew()}")

In [ ]:
# Configura o estilo do Seaborn
sns.set(style="whitegrid")

plt.figure(figsize=(12, 6))

# Boxplot para 'labor_avg'
plt.subplot(1, 2, 1)
sns.boxplot(y=data['labor_avg'], color='blue')
plt.title('Boxplot de labor_avg')

# Boxplot para 'birth year CEO'
plt.subplot(1, 2, 2)
sns.boxplot(y=data['birth_year'], color='green')
plt.title('Boxplot de birth year CEO')

plt.tight_layout()
plt.show()

## Tratamento de demais variáveis com MICE

Os demais valores ausentes são preenchidos utilizando o método MICE (Multiple Imputation by Chained Equations) com a biblioteca 'IterativeImputer' do sklearn.

In [ ]:
df_mice = data.copy()
df_mice.head()

In [ ]:
# Copia o df_mice para criar variáveis dummy
df_mice2 = df_mice.copy()
df_mice2 = pd.get_dummies(df_mice2)

In [ ]:
# Define Input de MICE e preenche NaN 

mice_imputer = IterativeImputer(estimator=linear_model.BayesianRidge(), n_nearest_features=None, imputation_order='ascending')
df_mice_imputed = pd.DataFrame(mice_imputer.fit_transform(df_mice2), columns=df_mice2.columns)

In [ ]:
df_mice_imputed.head()

In [ ]:
missing_data_3 = show_missing(df_mice_imputed).sort_values("pc_missing", ascending=False, ignore_index = True)

index_full_data_3 = list(missing_data_3[missing_data_3["missing"]==0].index)

missing_data_3.drop(labels=index_full_data_3, axis="index", inplace=True)

missing_data_3

In [ ]:
# Verifica dados imputados 

# MICE imputação
fig = plt.Figure()
null_values = data['labor_avg'].isnull() 
fig = df_mice_imputed.plot(x='sales', y='labor_avg', kind='scatter',
                           c=null_values, cmap='winter', s = 15,
                           title='MICE Imputation', colorbar=False)

In [ ]:
data.columns

In [ ]:
df_mice_imputed.columns

Reverte o processo de pd.get_dummies com base no df_mice_imputed

In [ ]:
# Dicionário que contém as colunas originais e suas respectivas colunas dummy
categorical_cols_transformed = {
    'exit_year': [col for col in df_mice_imputed.columns if 'exit_year' in col],
    'gender': [col for col in df_mice_imputed.columns if 'gender_' in col],
    'origin': [col for col in df_mice_imputed.columns if 'origin_' in col],
    'region_m': [col for col in df_mice_imputed.columns if 'region_m_' in col]
}

# Para cada coluna original, encontra a coluna dummy com o valor mais alto (1) e restaura a coluna original
for original_col, dummies in categorical_cols_transformed.items():
    df_mice_imputed[original_col] = df_mice_imputed[dummies].idxmax(axis=1).str.replace(original_col + "_", "")
    df_mice_imputed.drop(dummies, axis=1, inplace=True)

In [ ]:
df_mice_imputed.columns

## Feature Engineering
---

### Variável Dependente - Inatividade da Empresa
Chamamos "inativas" aquelas empresas que não apresentam vendas nos dois anos seguintes ao ano observado.\
**Nota:** ativa = 0, inativa = 1

In [ ]:
# Ordena o DataFrame por empresa e ano
df_mice_imputed.sort_values(by=['comp_id', 'year'], inplace=True)

# Cria colunas deslocadas para checar vendas nos dois anos seguintes
conditions_x1 = [((df_mice_imputed['comp_id'] == df_mice_imputed['comp_id'].shift(-1)) &
                  (df_mice_imputed['year'] == df_mice_imputed['year'].shift(-1) - 1)),

                 ((df_mice_imputed['comp_id'] != df_mice_imputed['comp_id'].shift(-1)) |
                  df_mice_imputed['year'] != df_mice_imputed['year'].shift(-1))]

values_x1 = [df_mice_imputed['sales'].shift(-1),
             np.nan]

df_mice_imputed['sales_x1'] = pd.Series(np.select(conditions_x1, values_x1)).fillna(0)


conditions_x2 = [
    ((df_mice_imputed['comp_id'] == df_mice_imputed['comp_id'].shift(-1)) &   
     (df_mice_imputed['year'] == df_mice_imputed['year'].shift(-1) - 2)),

    ((df_mice_imputed['comp_id'] == df_mice_imputed['comp_id'].shift(-2)) &
     (df_mice_imputed['year'] == df_mice_imputed['year'].shift(-2) - 2)),

    True
]

values_x2 = [df_mice_imputed['sales'].shift(-1),
             df_mice_imputed['sales'].shift(-2),
             np.nan]

df_mice_imputed['sales_x2'] = pd.Series(np.select(conditions_x2, values_x2)).fillna(0)


# Define condicao usada para identificar empresas que pararam de operar:
#  - sem vendas por mais de 2 anos
condition = ((df_mice_imputed['sales_x1'] == 0) & (df_mice_imputed['sales_x2'] == 0)) 

# Create a new 'dependente' column with 1 for ceased companies and 0 otherwise
df_mice_imputed['fechado'] = condition.astype(int)

Vamos conferir os valores da variavel dependente para algumas empresas do dataframe:

In [ ]:
# lista de colunas para avaliação
check_list = ['comp_id', 'year', 'exit_year', 'sales', 'sales_x1', 'sales_x2', 'fechado']

filtro = df_mice_imputed[check_list]
filtro_sub = filtro[(filtro['comp_id'] == 464021159936) | (filtro['comp_id'] == 1001541)]

filtro_sub

In [ ]:
# Remodela os dados de vendas para tratar valores negativos
for x in ['sales']:
    df_mice_imputed[x] = np.where(df_mice_imputed[x]<0, 0, df_mice_imputed[x])

df_mice_imputed['fechado'] = condition.astype(int)

In [ ]:
df_mice_imputed[check_list].describe()

Vamos averiguar agora os anos em que as empresas tiveram atividade e inatividade:

In [ ]:
# Converte "comp_id" para float antes de agrupar
df_mice_imputed['comp_id'] = df_mice_imputed['comp_id'].astype(float)

data_grouped = df_mice_imputed.groupby('comp_id')

# Conta anos de acompanhamento
comp_years = data_grouped['year'].count()

# Conta anos com vendas
sales_years = data_grouped.apply(lambda group: (group['sales'] > 0).sum())

# Conta anos sem vendas
no_sales_years = data_grouped.apply(lambda group: (group['sales'] == 0).sum())

# Conta anos de "inatividade"
inative_years = data_grouped.apply(lambda group: (group['fechado'] == 1).sum())

In [ ]:
pd.options.display.float_format = '{:.1f}'.format

activity_df = pd.DataFrame({'Total years':comp_years,
                            'Sales years':sales_years, 
                            'No sales years':no_sales_years,
                            'Inative years':inative_years}).reset_index()

activity_df['comp_id'] = activity_df['comp_id'].astype(float)
activity_df.head()

## Seleção de subset para análise
---
A modelagem é feita em cima de um subset dos dados com o objetivo de adicionar foco à analise e facilitar o processamento por estar trabalhando com menor volume de dados. O escopo desse subset pode ser adaptado iterativamente, atendendo a demandas que surjam na análise.

### Removendo dados do ano 2016
Registros do ano de 2016 são removidos do conjunto.

In [ ]:
data = data[data['year']!=2016].copy()
assert not any(data["year"] == 2016)

### Ano

In [ ]:
# Cria dataframe apenas com os dados do ano de 2012
data_2012 = df_mice_imputed[df_mice_imputed['year'] == 2012].copy()

assert data_2012.year.unique() == 2012 # verificando coluna de ano

In [ ]:
# Verifica valores da coluna "exit_year"
data_2012['exit_year'].value_counts()

In [ ]:
check_list = ['comp_id', 'year', 'exit_year', 'sales', 'sales_x1', 'sales_x2', 'fechado']

# Converte "-" para NaN
data_2012['exit_year'].replace('-', np.nan, inplace=True)

# Converte a coluna para float
data_2012['exit_year'] = data_2012['exit_year'].astype(float)

# Filtra o dataframe
filtro = data_2012[check_list]
filtro_sub = filtro[filtro['exit_year'] < 2012]
filtro_sub

Há na base empresas que encerram suas atividades antes de 2012 (de acordo com a variavel "exit_year") e apresentam vendas posteriormente, indicando uma inconsistência nos dados. Optamos por considerar o valor de "exit_year" e retiramos empresas para as quais "exit_year" < 2012.

Verifica as empresas que encerraram as atividades em 2012

In [ ]:
check_list = ['comp_id', 'year', 'exit_year', 'sales', 'sales_x1', 'sales_x2', 'fechado']

# Converte "-" para NaN
data_2012['exit_year'].replace('-', np.nan, inplace=True)

# Converte a coluna para float
data_2012['exit_year'] = data_2012['exit_year'].astype(float)

# Filtra o dataframe
filtro = data_2012[check_list]
filtro_sub = filtro[filtro['exit_year'] == 2012]
filtro_sub

In [ ]:
# Soma as colunas sales_x1 e sales_x2
total_sales_x1 = filtro_sub['sales_x1'].sum()
total_sales_x2 = filtro_sub['sales_x2'].sum()

print(f'Total de sales_x1: {total_sales_x1}')
print(f'Total de sales_x2: {total_sales_x2}')

Algumas empresas que encerram as atividades até 31/12/2012 apresentam vendas durante o ano de 2012 mas não no ano seguinte, conforme o esperado. Como nosso objetivo é prever as empresas que estam ativas em 31/12/2012 e tem suas atividades encerradas em até dois anos, as empresas que encerram as atividades em 2012 são retiradas da base.

In [ ]:
# Retira empresas que encerraram a atitidade até 2012, portanto não estam ativa na data de corte da análise
data_2012 = data_2012.copy()
data_2012 = data_2012[data_2012['exit_year'] > 2012]

# Verifica valores da coluna "exit_year"
data_2012['exit_year'].value_counts()

### Tamanho da Empresa

In [ ]:
# Considera apenas sales > 1000 e < 10000000
data_2012 = data_2012[(data_2012['sales'] > 1000) & (data_2012['sales'] < 10000000) ]

## Feature Engineering no Subset

### Assimetria

In [ ]:
# Filtra apenas colunas numéricas para calcular assimetria
numeric_cols = data_2012.select_dtypes(include=[np.number]).columns # Avaliar a simetria das variáveis

# Avalia a simetria das colunas numéricas
skewness_values = data_2012[numeric_cols].skew()

# Ordena os valores de assimetria do maior para o menor
sorted_skewness_values = skewness_values.sort_values(ascending=False)

# Mostra os valores de assimetria ordenados
print(sorted_skewness_values)

In [ ]:
# Identifica colunas com assimetria significativamente diferente de zero
cols_to_transform = skewness_values[skewness_values.abs() > 0.5].index
print(cols_to_transform)

Nas estatísticas descritivas abaixo, observa-se que a média é maior que o terceiro quartil, indicando uma distribuição bastante assimétrica. Criamos então uma coluna com o logarítmo de "sales" para auxiliar na análise.

In [ ]:
print(data_2012['sales'].describe())

A coluna log_sales mostra que, embora muitas empresas tenham vendas relativamente baixas (indicado por um logaritmo de vendas próximo ou igual a zero), há empresas que se destacam com vendas substancialmente mais altas. A transformação logarítmica ajuda a reduzir a assimetria e a concentrar os dados, tornando-os mais tratáveis para análise estatística e modelagem. No entanto, a presença de uma assimetria à esquerda ainda é evidente, sugerindo que muitas empresas no dataset têm vendas baixas ou nulas.

In [ ]:
# Função para criar coluna com logaritmo da coluna argumento

def criar_log(data_set, col):
    new_col = 'log_' + col
    data_set.loc[:,new_col] = data_set.loc[:,col]\
                                            .apply(lambda x: math.log(x)\
                                                   if x != 0\
                                                   else 0) 

In [ ]:
lista_de_colunas = ['sales']
new_cols = []
for col in lista_de_colunas:
    criar_log(data_2012, col)
    new_cols.append('log_' + col)

print(new_cols) 

#### "sales" vs "log_sales"

In [ ]:
# Cria histogramas
plt.figure(figsize=(12, 6))

# Histograma de sales
plt.subplot(1, 2, 1)
plt.hist(data_2012['sales'], bins=50, color='blue', edgecolor='black')
plt.title('Histograma de Sales')
plt.xlabel('Sales')
plt.ylabel('Frequência')

# Histograma de log_sales
plt.subplot(1, 2, 2)
plt.hist(data_2012['log_sales'], bins=50, color='green', edgecolor='black')
plt.title('Histograma de Log de Sales')
plt.xlabel('Log Sales')
plt.ylabel('Frequência')

plt.tight_layout()
plt.show()


#### Idade da Empresa

In [ ]:
data_2012['company_age'] = 2012 - data_2012['founded_year']

#### Outras Features

In [ ]:
# 2. Alavancagem Financeira:
#data_2012['financial_leverage'] = data_2012['curr_liab'] / data_2012['share_eq']

In [ ]:
# 3. Liquidez:
#data_2012['liquidity_ratio'] = data_2012['liq_assets'] / data_2012['curr_liab']

In [ ]:
# 4. Eficiência:
#data_2012['efficiency'] = data_2012['sales'] / data_2012['labor_avg']

In [ ]:
# 5. Tamanho da Empresa (exemplo baseado em vendas):
#sales_bins = [0, 1e6, 1e9, float('inf')]  # Exemplo de categorias: <1M, 1M-1B, >1B
#labels = ['small', 'medium', 'large']
#data_2012['company_size'] = pd.cut(data_2012['sales'], bins=sales_bins, labels=labels, right=False)

In [ ]:
# 5. Margem de Lucro:
#data_2012['profit_margin'] = data_2012['profit_loss_year'] / data_2012['sales']

## Limpeza Final
Como ultima manipulação dos dados, retiramos variáveis usadas na limpeza que não serão usadas na modelagem.

In [ ]:
data_2012 = data_2012.drop(columns=['sales_x1','sales_x2', 'exit_year', 'year'])


## Matriz de correlação

In [ ]:
# Lista todas as colunas exceto "dependente"
cols = [col for col in data_2012.columns if col != 'fechado']

# Dividi as colunas em dois grupos
half = len(cols) // 2
group1 = cols[:half] + ['fechado']
group2 = cols[half:] + ['fechado']

# Considera apenas colunas numéricas
group1 = [col for col in group2 if data_2012[col].dtype in ['int64', 'float64']]
group2 = [col for col in group2 if data_2012[col].dtype in ['int64', 'float64']]

# Cria as duas matrizes de correlação
correlation_matrix1 = data_2012[group1].corr()
correlation_matrix2 = data_2012[group2].corr()

# Visualiza a primeira matriz de correlação
plt.figure(figsize=(16,10))
sns.heatmap(correlation_matrix1, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix - Group 1')
plt.show()

# Visualiza a segunda matriz de correlação
plt.figure(figsize=(16,10))
sns.heatmap(correlation_matrix2, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix - Group 2')
plt.show()

In [ ]:
missing_data_4 = show_missing(data_2012).sort_values("pc_missing", ascending=False, ignore_index = True)

#full_index = list(missing_data_4[missing_data_4["missing"]==0].index)

#missing_data_4.drop(labels=full_index, axis="index", inplace=True)

missing_data_4.sort_values(by="variable", ignore_index=True)

Confirmamos que o banco de dados final gerado está completo, sem dados faltando através do processo de limpeza com alterações e remoções.

### Análise por agrupamento de variável dependente:
---

Vamos Agrupar as empresas marcadas pela variável dependente (fechada) para buscar visualizar algumas das tendências das empresas que fecharam no ano de 2012:

In [ ]:
groups_2012 = data_2012.groupby(by="fechado")
# fechado = 1, aberto = 0

df_groups = groups_2012.agg(['mean', 'median'])

df_groups


In [ ]:
sns.boxplot(data=data_2012, x='fechado', y='company_age', 
            showfliers = False, medianprops = {"color":"red"})

# Configuração do gráfico:
plt.xlabel('Status de Operação (fechada)')
plt.ylabel('Idade da empresa')
plt.title('Idade de empresas fechadas vs abertas em 2012')

# Show the plot
plt.show()

print("valores médios de idade da empresa:")
print(df_groups["company_age"]['mean'])

In [ ]:
def group_box(column, data = data_2012):
    ''' Funcao para rodar boxplot agrupado '''
    sns.boxplot(data=data, x='fechado', y=column, 
            showfliers = False, medianprops = {"color":"red"})

    # Configuração do gráfico:
    plt.xlabel('Status de Operação (fechada)')
    plt.ylabel(f'Valor variável "{column}"')
    plt.title('Análise agrupada {column}')

    # Show the plot
    plt.show()

    #print("valores médios da variável:")
    #print(df_groups[column]['mean'])


In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

# Create histograms for each group using Seaborn
sns.histplot(data=data_2012, x='company_age', hue='fechado', bins=20, alpha=0.5, ax=ax, kde=True)

# Set labels and title
ax.set_xlabel('Idade da empresa')
ax.set_ylabel('Frequencia da Idade')
ax.set_title('Histograma da Idade das empresas')

# Show the plot
plt.show()

In [ ]:

def group_hist(column, data=data_2012):
    ''' Funcao para gerar o histograma com linha de tendencia agrupado'''
    
    fig, ax = plt.subplots(figsize=(10, 6))

    # Create histograms for each group using Seaborn
    sns.histplot(data=data, x=column, hue='fechado', bins=20, alpha=0.5, ax=ax, kde=True)

    # Set labels and title
    ax.set_xlabel(f'valor da variável {column}')
    ax.set_ylabel(f'Frequência da variável {column}')
    ax.set_title(f'Histograma de {column}')

    # Show the plot
    plt.show()

Conseguimos observar que a mediana entre as empresas que fecharam e ficaram abertas é a mesma, assim como seus quartis inferiores, e ambas possuem uma variação alta nas empresas com idade acima da mediana. 

Assim podemos assumir que maiores idades podem ser um dos indicadores de maiores chances de não fechamento e empresas mais novas como indicadores de maior probabilidade de fechamento, principalmente para casos de 0 anos de idade.

Porém não demonstra forte diferença entre os dois cenários para ser uma variável forte para a predição.

#### Analisando amortização:

In [ ]:
group_box("amort")
group_hist("amort")

Vamos converter para log para conseguirmos analisar melhor as tendências

In [ ]:
criar_log(data_2012, "amort")

group_box("log_amort")
group_hist("log_amort")

Conseguimos visualizar que a amortização possui valor mediano e distribuição quase identica entre os casos de "fechado", possuindo também uma distribuição de frequência dos valores, quase identico com linhas de regressão extremamente similares visualmente.

Resalve para o caso de log_amort com valores de 0 onde há maior probabilidade de ser "aberto" dada a proporção. 

Com uma leve tendência de ser fechada conforme os valores de amortização sobem passando de 6.

Desta forma podemos notar uma tendência leve por empresas abertas para menores valores de amortização (log_amortização).

#### Analisando Indústria:

In [ ]:
group_hist("ind")

Podemos verificar uma tendência de empresas da industria 1 e 2 de estarem abertas em relação a empresas da industria 3, indicando que há uma menor probabilidade nos dados de uma empresa fechar se for do tipo 1 ou 2.

Vamos confirmar esta tendência verificando a proporção por indústria.

In [ ]:
def proportion_1(column, data=data_2012):
    '''Funcao retorna proporcao de fechado para cada caso'''

    industry_effect = data_2012.groupby(["fechado", column])[column].count()

    pivot_table = data_2012.pivot_table(index=column, columns='fechado', aggfunc='size', fill_value=0)

    # calcular proporção
    pivot_table['Proportion'] = pivot_table[1] / (pivot_table[0] + pivot_table[1])

    # Reset the index to make 'ind' a regular column
    # Resetando index para fazer "ind" uma coluna normal
    pivot_table = pivot_table.reset_index()

    # Renomeando por clareza
    pivot_table.columns = [column, '0_Count', '1_Count', 'Proporcao_1']

    return pivot_table

In [ ]:
print(proportion_1("ind"))

Analisando as proporções conseguimos confirmar que na verdade a tendência real é de a empresa estar aberta se for do tipo 1 ou 2 com 37% e 41% de fechamento nestes casos enquanto industria 3 pode não ser um bom representante com uma relação próxima a 50% (52% entre fechadas e abertas).

In [ ]:
data_2012.drop(columns = "amort")

Numa análise mais minuciosa destas variáveis conseguimos ver seu valor para modelos de predição baseado em distribuição e proporção dos valores, porém não aparentam ser as variáveis mais fortes para um modelo de predição.

### Seleção de variáveis pelo método Stepwise:
---
Primeiro vamos verificar o formato das variáveis

In [ ]:
#Tipos de variáveis do dataframe
print(list(data_2012.dtypes.unique()))

print(data_2012.select_dtypes(include='O'))

col_obj = (list(data_2012.select_dtypes(include='O')))
print(col_obj)

Para aplicar o modelo precisamos de todas as variáveis em modelo numérico:

In [ ]:
# Convertendo as colunas de objetos em dummies [0 ou 1]
data_2012_num = pd.get_dummies(data_2012, columns=col_obj, dtype="uint8")

#data_2012_num.dtypes.unique()

print(list(data_2012_num.select_dtypes(include='uint8')))

Vamos realizar um fit de modelos OLS (ordinary least squares) de regressão.

Depois removemos as variáveis com os maiores valores de p-value do modelo (maior que 0.05).

In [ ]:
import statsmodels.api as sm

# X é a matrix de dados de variáveis preditoras
X = data_2012_num.drop(columns=["fechado"])
# y é nossa variável de predição ("fechado" variável dependente)
y = data_2012_num["fechado"]


# Adiciona uma constante (intercept) na matrix de variáveis
X = sm.add_constant(X)

# Fit do modelo inicial
model = sm.OLS(y, X).fit()

In [ ]:
# Realiza seleção stepwise:
selected_features = model.model.exog_names[:]
while True:
    prev_model = model
    p_values = model.pvalues[1:]  # Exclude the constant term
    feature_to_remove = p_values.idxmax()
    if p_values[feature_to_remove] > 0.05:
        break
    selected_features.remove(feature_to_remove)
    X = X[selected_features]
    model = sm.OLS(y, X).fit()

# Final selected features
print("Variáveis selecionadas:", selected_features)
print(f"Quantidade de variáveis selecionadas: {len(selected_features)}")
print(f"Total de variáveis: {len(data_2012_num.columns)}")


In [ ]:
# Resumo do modelo final
summary = model.summary()
summary.tables[1]


In [ ]:
# Pegando a tabela de valores dos coeficientes:
coefficients = summary.tables[1]

# Convertendo objeto para dataframe:
coefficients_df = pd.DataFrame(coefficients.data[1:], columns=coefficients.data[0])

# Renomeando últimas duas colunas para facilitar chamada:
coefficients_df.rename(columns={"[0.025":"lower", "0.975]":"upper"}, inplace=True)

#convertendo dados para numérico:
coefficients_df[['std err', 't', 'lower', 'upper']] = coefficients_df[
    ['std err', 't', 'lower', 'upper']
].apply(pd.to_numeric)


# Convertendo coef e p-value para floats individualmente:
coefficients_df[['coef', 'P>|t|']] = coefficients_df[['coef', 'P>|t|']].astype(float)

# Configurando display de float:
pd.set_option('display.float_format', '{:,.8f}'.format)

#display(coefficients_df)

# Variáveis significantes:
significant_features = coefficients_df[coefficients_df['P>|t|'] < 0.05]

Agora vamos ordenar por significancia nossas variáveis e verificar os resultados:

In [ ]:
sorted_features = significant_features.sort_values(by='coef', ascending=False)

#Trocando o nome vazio por "var" de variável
sorted_features.rename(columns={'':'var'}, inplace=True) 

display(sorted_features)

list_features = list(sorted_features['var'].unique())
print(list_features)
print(len(list_features))

Variáveis como "comp_id" número da impressa não deveriam interferir na predição pois são valores arbitrarios de registro de cada empresa.

## Exportação da base tratada para modelagem

In [ ]:
data_2012.to_csv("data_2012.csv", index=False)
data_2012_num.to_csv("data_2012_numerical.csv", index=False)

In [ ]:
data_2012.columns

In [ ]:
data_2012_num.columns

In [ ]:
#data_2012.to_excel("data_2012.xlsx")